<a href="https://colab.research.google.com/github/ohilikeit/DSF_AED/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
os.chdir('/content/drive/MyDrive/ML_projects/DSF')

df = pd.read_csv('./자동심장 충격기 정보 조회(AED)(표준 데이터).csv', encoding='cp949')
df['address'] = df['설치기관주소'].apply(lambda x: x.split(' ')[1])
df_dongdaemoon = df[df['address'] == "동대문구"]

# 1. 인구 데이터 - 행정동 별 수요점수 계산

In [2]:
pop = pd.read_csv('./동대문구_동별_연령대별_인구수.csv')

In [4]:
cols = pop.columns
pop['male'] = pop[cols[2]] * 0.01 + pop[cols[3]] * 0.026 + pop[cols[4]] * 0.039 + pop[cols[5]] * 0.075 + pop[cols[6]] * 0.139 + pop[cols[7]] * 0.7
pop['male'] = pop['male'] / pop['male'].max()
pop['female'] = pop[cols[8]] * 0.01 + pop[cols[9]] * 0.026 + pop[cols[10]] * 0.039 + pop[cols[11]] * 0.075 + pop[cols[12]] * 0.139 + pop[cols[13]] * 0.7
pop['female'] = pop['female'] / pop['female'].max()
pop['w'] = 0.64 * pop['male'] + 0.36 * pop['female']

,행정동코드명,총상주인구수,남성연령대10상주인구수,남성연령대20상주인구수,남성연령대30상주인구수,남성연령대40상주인구수,남성연령대50상주인구수,남성연령대60이상상주인구수,여성연령대10상주인구수,여성연령대20상주인구수,여성연령대30상주인구수,여성연령대40상주인구수,여성연령대50상주인구수,여성연령대60이상상주인구수,male,female,w
0,용신동,30305,1543,2695,2733,2353,2536,3708,1397,2743,2406,1966,2221,4004,0.489269,0.461159,0.479149
1,제기동,24837,1317,2261,1882,1768,2039,3444,1221,1886,1515,1463,1908,4133,0.438450,0.454489,0.444224
2,전농동,50674,3743,3609,3585,3987,4181,5896,3741,3494,3423,3897,4184,6934,0.778669,0.803372,0.787562
3,답십리동,58632,4519,3879,4888,4702,4651,6002,4317,4060,4980,4494,4837,7303,0.816840,0.867090,0.834930
4,장안동,69413,5685,4439,5652,6117,5422,7391,5249,4365,5474,5751,5460,8408,1.000000,1.000000,1.000000
5,청량리동,21250,1162,1353,1371,1488,1932,3353,1073,1184,1173,1310,1876,3975,0.417110,0.433083,0.422860
6,회기동,10023,568,1349,765,666,564,850,609,1491,738,643,599,1181,0.117120,0.138401,0.124781
7,휘경동,36888,2408,4034,2633,2651,2744,3877,2413,3792,2288,2595,2755,4698,0.520118,0.547008,0.529799
8,이문동,39400,2773,3672,2598,3010,3045,4327,2708,3590,2321,2963,3013,5380,0.575679,0.619419,0.591425


# 2. 도보 노드 데이터

In [ ]:
dobo = pd.read_csv('./서울시 자치구별 도보 네트워크 공간정보.csv', encoding='cp949')

df_dobo = dobo[(dobo['노드링크 유형'] == 'NODE') &
               (dobo['시군구명'] == '동대문구') &
               (dobo['육교'] == 0) &
               (dobo['횡단보도'] == 0)]

# 용두동 + 신설동 = 용신동
df_dobo.loc[(df_dobo['읍면동명'] == "용두동") |
            (df_dobo['읍면동명'] == "신설동"), "읍면동명"] = "용신동"
df_dobo = df_dobo[['노드 WKT', '노드 ID', '시군구명', '읍면동명']].reset_index(drop=True)

# point 추출
def point(txt):
    a = re.findall(r'\d+', txt)
    return float(a[0] + '.' + a[1]), float(a[2] + '.' + a[3])

df_dobo['lon'] = df_dobo['노드 WKT'].apply(lambda x: point(x)[0])
df_dobo['lat'] = df_dobo['노드 WKT'].apply(lambda x: point(x)[1])

# 수요 가중치(인구) 병합
pop[['행정동코드명', 'w']]
df_dobo = pd.merge(df_dobo, pop[['행정동코드명', 'w']], how= 'left', right_on = '행정동코드명', left_on = '읍면동명')
del df_dobo['행정동코드명']

,노드 WKT,노드 ID,시군구명,읍면동명,lon,lat,w
0,POINT(127.05837146633299 37.59439986547229),92202,동대문구,이문동,127.058371,37.594400,0.591425
1,POINT(127.05407178305668 37.59294112861541),86075,동대문구,회기동,127.054072,37.592941,0.124781
2,POINT(127.05424994824688 37.592735080131284),86074,동대문구,회기동,127.054250,37.592735,0.124781
3,POINT(127.05374092778224 37.59278794893249),86069,동대문구,회기동,127.053741,37.592788,0.124781
4,POINT(127.04949572410594 37.591444728097464),87176,동대문구,회기동,127.049496,37.591445,0.124781
...,...,...,...,...,...,...,...
8536,POINT(127.03956543808944 37.57744214399052),72386,동대문구,용신동,127.039565,37.577442,0.479149
8537,POINT(127.03937278548555 37.5774467854538),72387,동대문구,용신동,127.039373,37.577447,0.479149
8538,POINT(127.03955675484558 37.577334586208885),72388,동대문구,용신동,127.039557,37.577335,0.479149
8539,POINT(127.0393727324653 37.57734609035601),72389,동대문구,용신동,127.039373,37.577346,0.479149
